# Imports

In [23]:
# Basics
import pandas as pd
import numpy as np
from pydataset import data

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Metric Tools
import sklearn.linear_model as skl_lm
import sklearn.feature_selection as skl_fs
import sklearn.preprocessing as skl_pp

# Custom
import wrangle as wrg

# Exercise 1.1

- Load the tips dataset. Create a column named price_per_person. This should be the total bill divided by the party size.

In [2]:
df = data('tips')
df.head(3)

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3


In [3]:
# Rename Size (.size is an attribute)

df = df.rename(columns={'size':'number_of_people'})

In [4]:
# Add price per person to the nearest 2 decimals

df['price_per_person'] = round((df.total_bill / df.number_of_people), 2)
df.head(2)

,total_bill,tip,sex,smoker,day,time,number_of_people,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,3.45


In [5]:
# A bit more data prep

cat_columns = ['sex', 'day', 'time', 'smoker']

for col in cat_columns:
    tips_dummy = pd.get_dummies(df[col],
                                    prefix=df[col].name,
                                    dummy_na=False,
                                    drop_first = True)
    df = pd.concat([df, tips_dummy], axis=1)
    df = df.drop(columns=[col])
    

In [6]:
df.head(2)

,total_bill,tip,number_of_people,price_per_person,sex_Male,day_Sat,day_Sun,day_Thur,time_Lunch,smoker_Yes
1,16.99,1.01,2,8.49,0,0,1,0,0,0
2,10.34,1.66,3,3.45,1,0,1,0,0,0


In [7]:
# For good measure, lets split the data as well

train, validate, test = wrg.split_data(df)

train <> (136, 10)
validate <> (59, 10)
test <> (49, 10)


In [8]:
# X's and Y's
y = 'tip'

# split train into X (dataframe, drop target) & y (series, keep target only)
x_train = train.drop(columns=[y])
y_train = train[y]

# split validate into X (dataframe, drop target) & y (series, keep target only)
x_validate = validate.drop(columns=[y])
y_validate = validate[y]

# split test into X (dataframe, drop target) & y (series, keep target only)
x_test = test.drop(columns=[y])
y_test = test[y]

x_train.head(1)

,total_bill,number_of_people,price_per_person,sex_Male,day_Sat,day_Sun,day_Thur,time_Lunch,smoker_Yes
195,16.58,2,8.29,1,0,0,1,1,1


In [9]:
# Scale data as well

# Make it
scaler = skl_pp.MinMaxScaler()

# Fit it
scaler.fit(x_train)

# Use it
x_train_scaled = scaler.transform(x_train)
x_validate_scaled = scaler.transform(x_validate)
x_test_scaled = scaler.transform(x_test)

# Exercise 1.2

- Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? 

#### I would guess the total bill and number of people would be the largest predictors

---------------------------------------------------

# Exercise 1.3

- Use select k best to select the top 2 features for predicting tip amount. What are they?

In [10]:
# Tip via select-k-best

# Make it
kbest = skl_fs.SelectKBest(skl_fs.f_regression, k=2)

# Fit it
kbest.fit(x_train, y_train)

# Use it
kbest_features = x_train.columns[kbest.get_support()].tolist()


print(f'The two best features to build our model on will be', kbest_features)

The two best features to build our model on will be ['total_bill', 'number_of_people']


# Exercise 1.4

- Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [29]:
# Tips features via RFE

# Make it
lm = skl_lm.LinearRegression()
rfe = skl_fs.RFE(lm, n_features_to_select=2)

# Fit it
rfe.fit(x_train, y_train)

# Use it
rfe_columns = x_train.columns[rfe.support_].tolist()

print(f'According to RFE, the two best features to build our model on will be', rfe_columns)

According to RFE, the two best features to build our model on will be ['day_Sat', 'day_Sun']


# Exercise 1.5

- Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

-------------------------------------------

#### Both feature selection methods chose features based on different priorities.  The features chosen do change slightly when chosing the number of features

In [32]:
# [WITH 3 FEATURES] Tips features via RFE

# Make it
lm = skl_lm.LinearRegression()
rfe = skl_fs.RFE(lm, n_features_to_select=3)

# Fit it
rfe.fit(x_train, y_train)

# Use it
rfe_columns = x_train.columns[rfe.support_].tolist()

print(f'According to RFE, the two best features to build our model on will be', rfe_columns)

According to RFE, the two best features to build our model on will be ['total_bill', 'day_Sat', 'day_Sun']


In [33]:
# [WITH 4 FEATURES] Tips features via RFE

# Make it
lm = skl_lm.LinearRegression()
rfe = skl_fs.RFE(lm, n_features_to_select=4)

# Fit it
rfe.fit(x_train, y_train)

# Use it
rfe_columns = x_train.columns[rfe.support_].tolist()

print(f'According to RFE, the two best features to build our model on will be', rfe_columns)

According to RFE, the two best features to build our model on will be ['total_bill', 'number_of_people', 'day_Sat', 'day_Sun']


# Exercies 2

- Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [22]:
# Build the function

def select_kbest(x, y, k):
    '''
    Takes in an x and y dataframe and the number of features to select
    and returns the names of the top features based on SelectKBest
    '''
    # Make it
    kbest = skl_fs.SelectKBest(skl_fs.f_regression, k=k)

    # Fit tit
    kbest.fit(x, y)
    
    # Use it 
    return x.columns[kbest.get_support()].tolist()

select_kbest(x_train, y_train, 2)

['total_bill', 'number_of_people']

# Exercise 3

- Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [36]:
# Build the function

def rfe(x, y, k):
    '''
    Takes in the predictors, the target, and the number of features to select,
    and it should return the top k features based on the RFE class. 
    '''
    # Make it
    lm = sklearn.linear_model.LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)

    # Fit it
    rfe.fit(x, y)
    
    # Use it
    features_to_use = x.columns[rfe.support_].tolist()
    
    #
    #all_rankings = show_features_rankings(x, rfe)
    
    return features_to_use

In [37]:
rfe(x_train, y_train, 3)

['total_bill', 'day_Sat', 'day_Sun']

In [40]:
def rfe_feature_rankings(x, y, k):
    '''
    Takes in the predictors, the target, and the number of features to select,
    and it should return a database of the features ranked by importance
    '''
    
    # Make it
    lm = sklearn.linear_model.LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)

    # Fit it
    rfe.fit(x, y)
    
    var_ranks = rfe.ranking_
    var_names = x_train.columns.tolist()
    ranks = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    ranks = ranks.sort_values(by="Rank", ascending=True)
    return ranks

In [41]:
rfe_feature_rankings(x_train, y_train, 2)

,Var,Rank
4,day_Sat,1
5,day_Sun,1
0,total_bill,2
1,number_of_people,3
2,price_per_person,4
7,time_Lunch,5
8,smoker_Yes,6
6,day_Thur,7
3,sex_Male,8


# Exercise 4

- Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [38]:
df = data('swiss')
df.head(3)

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
